# group FEMA dataset by (zipcode, year, month)

In [1]:
import os
import numpy as np
from astropy.table import Table
import astropy.table as aTable
import matplotlib.pyplot as plt

In [2]:
fema = Table.read('/Users/chahah/data/noah/fema.trimed.csv', format='csv')

In [3]:
fema[:2]

agricultureStructureIndicator,asOfDate,baseFloodElevation,basementEnclosureCrawlspace,reportedCity,condominiumIndicator,policyCount,countyCode,communityRatingSystemDiscount,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,censusTract,floodZone,houseWorship,latitude,longitude,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,state,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,reportedZipcode,primaryResidence,id,amountPaidOnTotalClaim,totalTotalInsuranceCoverage
int64,str24,int64,int64,str30,str1,int64,int64,int64,str24,int64,int64,int64,int64,str3,int64,float64,float64,int64,float64,int64,int64,int64,int64,int64,str24,str24,float64,float64,float64,int64,str1,int64,str2,int64,int64,int64,int64,int64,str24,float64,int64
0,2020-08-23T02:35:07.366Z,9,--,ST PETERSBURG,N,1,12103,5,1996-10-08T04:00:00.000Z,0,3,93,--,AE,0,27.8,-82.6,--,--,101,1,0,10,1,1959-07-23T04:00:00.000Z,1995-03-29T05:00:00.000Z,21567.68,32800.0,--,0,1,0,FL,65500,32800,1996,33703,1,5f6a41bce1e65249b3237e5f,54367.68,98300
0,2020-08-23T02:35:07.366Z,--,--,RIVER RIDGE,N,1,22051,5,1995-05-08T04:00:00.000Z,0,--,--,--,X,0,30.0,-90.2,--,--,--,1,0,10,1,1960-06-15T04:00:00.000Z,1995-03-31T05:00:00.000Z,18987.7,5897.3,--,0,7,0,LA,75000,18000,1995,70123,0,5f6a41bce1e65249b3237e60,24885.0,93000


# CRS treatment status
- CRS scores <= 9 will be considered treated
- CRS score > 9 will be considered control

`crs_treat` will be used to estimate the average CRS treatment status for a zipcode

In [4]:
# CRS scores <= 9 will be considered treated
crs_treat = np.zeros(len(fema)).astype(bool)
crs_treat[fema['communityRatingSystemDiscount'] <= 9] = True

In [5]:
fema['crs_treat'] = crs_treat

# remove entries without zipcodes
**we should check that the removed entries aren't biased in some what**

In [6]:
print('%i of %i entries do not have zipcodes' % (np.sum(fema['reportedZipcode'].mask), len(fema['reportedZipcode'])))
print('%.2f percent' % (np.mean(fema['reportedZipcode'].mask)*100.))

44718 of 1916033 entries do not have zipcodes
2.33 percent


In [7]:
fema = fema[~fema['reportedZipcode'].mask]

In [8]:
weird = (
    ((fema['reportedZipcode'] == 77571) & (fema['state'] == 'NJ')) | 
    (fema['reportedZipcode'] == 0) |
    (fema['reportedZipcode'] == 99999)
)
fema = fema[~weird]

In [9]:
print('%i of %i are weird' % (np.sum(weird), len(weird)))
print('%.2f percent' % (np.mean(weird) * 100))

23 of 1871315 are weird
0.00 percent


# group by [zipcode, yearOfLoss, monthOfLoss]
State is mainly to take care of spurious zipcodes

In [14]:
months = np.array([int(date.split('-')[1]) for date in fema['dateOfLoss']])

In [15]:
fema['monthOfLoss'] = months

In [16]:
fema_zip = fema['reportedZipcode', 'yearOfLoss', 'monthOfLoss', 'state', 'longitude', 'latitude', 'crs_treat', 'communityRatingSystemDiscount', 'primaryResidence'].group_by(['reportedZipcode', 'yearOfLoss', 'monthOfLoss'])

In [17]:
# fill in missing longitude or latitude
N_nolonglat = 0
for group in fema_zip.groups:
    if np.sum(group['longitude'].mask) < len(group):
        group['longitude'][group['longitude'].mask] = group['longitude'][~group['longitude'].mask][0]
        group['latitude'][group['latitude'].mask] = group['latitude'][~group['latitude'].mask][0]            
    elif np.sum(~group['longitude'].mask) == 0: 
        N_nolonglat += 1

In [18]:
print('%i or %i entries do not have long,lat data' % (N_nolonglat, len(fema_zip))) 
print('%.2f percent'% (100*float(N_nolonglat) / float(len(fema_zip))))

4093 or 1871292 entries do not have long,lat data
0.22 percent


In [19]:
# manual corrections to zipcode -- state mismatch 
fema_zip['state'][(fema_zip['reportedZipcode'] == 10011) & (fema_zip['state'] == 'LA')] = 'NY'
fema_zip['state'][(fema_zip['reportedZipcode'] == 10035) & (fema_zip['state'] == 'VA')] = 'NY'
fema_zip['state'][(fema_zip['reportedZipcode'] == 11720) & (fema_zip['state'] == 'PA')] = 'NY'
fema_zip['state'][(fema_zip['reportedZipcode'] == 11721) & (fema_zip['state'] == 'PA')] = 'NY'
fema_zip['state'][(fema_zip['reportedZipcode'] == 27546) & (fema_zip['state'] == 'WV')] = 'NC'
fema_zip['state'][(fema_zip['reportedZipcode'] == 31602) & (fema_zip['state'] == 'FL')] = 'GA'
fema_zip['state'][(fema_zip['reportedZipcode'] == 37064) & (fema_zip['state'] == 'KY')] = 'TN'
fema_zip['state'][(fema_zip['reportedZipcode'] == 38127) & (fema_zip['state'] == 'TX')] = 'TN'
fema_zip['state'][(fema_zip['reportedZipcode'] == 38901) & (fema_zip['state'] == 'MI')] = 'MS'
fema_zip['state'][(fema_zip['reportedZipcode'] == 39307) & (fema_zip['state'] == 'AL')] = 'MS'
fema_zip['state'][(fema_zip['reportedZipcode'] == 56219) & (fema_zip['state'] == 'SD')] = 'MN'
fema_zip['state'][(fema_zip['reportedZipcode'] == 58105) & (fema_zip['state'] == 'WI')] = 'ND'
fema_zip['state'][(fema_zip['reportedZipcode'] == 63633) & (fema_zip['state'] == 'OH')] = 'MO'
fema_zip['state'][(fema_zip['reportedZipcode'] == 5155) & (fema_zip['state'] == 'UT')] = 'VT'
fema_zip['state'][(fema_zip['reportedZipcode'] == 10000) & (fema_zip['state'] == 'PA')] = 'NY'
fema_zip['state'][(fema_zip['reportedZipcode'] == 19342) & (fema_zip['state'] == 'NJ')] = 'PA'
fema_zip['state'][(fema_zip['reportedZipcode'] == 65251) & (fema_zip['state'] == 'MI')] = 'MO'
fema_zip['state'][(fema_zip['reportedZipcode'] == 70535) & (fema_zip['state'] == 'MA')] = 'LA'

In [20]:
# fix state mismatches
for group in fema_zip.groups: 
    if np.sum(group['state'] != group['state'][0]): 
        is_un = (group['state'] == 'UN')
        group['state'][is_un] = group['state'][~is_un][0]
        
        uniq_state, n_state = np.unique(group['state'], return_counts=True)
        assert np.sum(n_state == n_state[np.argmax(n_state)]) == 1, group['reportedZipcode', 'state']
        group['state'][group['state'] != uniq_state[np.argmax(n_state)]] = uniq_state[np.argmax(n_state)]
        
        assert np.sum(group['state'] != group['state'][0]) == 0, group

In [21]:
fema_zip = fema_zip.group_by(['reportedZipcode', 'yearOfLoss', 'monthOfLoss', 'state'])

In [22]:
# check that all the states match 
for group in fema_zip.groups: 
    assert np.sum(group['state'] != group['state'][0]) == 0

In [23]:
# combine each group with mean
fema_zip_group = fema_zip.groups.aggregate(np.mean)

/opt/anaconda3/envs/noah/lib/python3.7/site-packages/astropy/table/groups.py:259: UserWarning: Warning: converting a masked element to nan.
  vals = np.array([func(par_col[i0: i1]) for i0, i1 in zip(i0s, i1s)])


In [24]:
fema_zip_group[::10000][:15]

reportedZipcode,yearOfLoss,monthOfLoss,state,longitude,latitude,crs_treat,communityRatingSystemDiscount,primaryResidence
int64,int64,int64,str2,float64,float64,float64,float64,float64
601,1992,1,PR,-66.7,18.2,0.0,11.0,0.0
4063,1996,10,ME,-70.4,43.5,1.0,7.0,0.4666666666666667
7946,1978,1,NJ,-74.5,40.7,0.5,9.0,0.0
11362,1987,7,NY,-73.7,40.800000000000004,0.0,11.0,0.0
14738,1998,1,NY,-79.2,42.0,0.0,11.0,1.0
19010,1999,9,PA,-75.4,40.0,0.0,11.0,1.0
23601,2012,10,VA,-76.5,37.0,0.0,11.0,1.0
28205,2003,11,NC,-80.8,35.2,1.0,4.0,1.0
32019,1982,6,FL,-81.0,29.133333333333336,1.0,5.0,0.0


In [25]:
fema_zip_insurance = fema['reportedCity', 
                          'reportedZipcode', 
                          'yearOfLoss', 
                          'monthOfLoss',  
                          'policyCount', 
                          'amountPaidOnBuildingClaim', 
                          'amountPaidOnContentsClaim', 
                          'totalBuildingInsuranceCoverage', 
                          'totalContentsInsuranceCoverage', 
                          'amountPaidOnTotalClaim', 
                          'totalTotalInsuranceCoverage'].group_by(['reportedZipcode', 'yearOfLoss', 'monthOfLoss'])

In [26]:
# add up all the insurance claims for the zipcode per year
fema_zip_insurance_group = fema_zip_insurance.groups.aggregate(np.sum)

In [27]:
fema_zip_insurance_group[::10000][:15]

reportedZipcode,yearOfLoss,monthOfLoss,policyCount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,amountPaidOnTotalClaim,totalTotalInsuranceCoverage
int64,int64,int64,float64,float64,float64,int64,int64,float64,int64
601,1992,1,1.0,1106.92,0.0,14600,0,1106.92,14600
4063,1996,10,15.0,96808.1,20319.410000000003,1291400,249500,117127.51000000001,1540900
7946,1978,1,2.0,452.95,270.0,17300,2800,722.95,20100
11362,1987,7,6.0,19901.72,0.0,144000,43100,19901.72,187100
14738,1998,1,1.0,4532.0,0.0,68200,0,4532.0,68200
19010,1999,9,2.0,14410.16,820.25,200800,19800,15230.41,220600
23601,2012,10,1.0,4123.64,0.0,234000,11600,4123.64,245600
28205,2003,11,1.0,4430.83,0.0,30000,8000,4430.83,38000
32019,1982,6,3.0,1873.1399999999999,0.0,173100,24100,1873.1399999999999,197200


In [28]:
fema_zip_comb = aTable.join(fema_zip_group, fema_zip_insurance_group, keys=['reportedZipcode', 'yearOfLoss', 'monthOfLoss'], join_type='left')

In [29]:
fema_zip_comb

reportedZipcode,yearOfLoss,monthOfLoss,state,longitude,latitude,crs_treat,communityRatingSystemDiscount,primaryResidence,policyCount,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,amountPaidOnTotalClaim,totalTotalInsuranceCoverage
int64,int64,int64,str2,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,int64
601,1992,1,PR,-66.7,18.2,0.0,11.0,0.0,1.0,1106.92,0.0,14600,0,1106.92,14600
601,1992,10,PR,nan,nan,0.0,11.0,0.0,2.0,764.89,4731.0,14000,9000,5495.889999999999,23000
601,1993,5,PR,nan,nan,0.0,11.0,0.0,4.0,3131.57,6839.5,14000,30500,9971.07,44500
601,1998,9,PR,-66.7,18.2,0.0,11.0,0.0,1.0,32150.0,0.0,500000,0,32150.0,500000
601,2003,11,PR,-66.7,18.2,0.0,11.0,1.0,2.0,4719.68,3318.5,20000,7800,8038.179999999999,27800
601,2008,9,PR,-66.7,18.2,0.0,11.0,1.0,1.0,3821.27,1989.0,10600,15000,5810.27,25600
601,2008,10,PR,-66.7,18.2,0.0,11.0,1.0,1.0,2360.14,1117.0,10600,15000,3477.14,25600
601,2017,9,PR,-66.7,18.2,0.0,11.0,0.0,1.0,8540.26,25424.25,17300,100000,33964.51,117300
602,1988,4,PR,nan,nan,0.0,11.0,0.0,1.0,0.0,20824.0,0,50000,20824.0,50000


In [30]:
fema_zip_comb.write('/Users/chahah/data/noah/fema.zipcode.csv', format='csv', overwrite=True)